In [1]:
from utils import psnr_ssim_rect, unwrap,  load_images, cal_psnr_ssim
import numpy as np
import os
import cv2

In [2]:
cwd = os.getcwd()
print(cwd)
num, files = load_images(cwd)
print('number of files: ', num)

/Users/vanple/Desktop/ucla/_Courses_/3D-vis-reasoning/stage2/stage2_rectification
number of files:  11


In [3]:
# grid search for the best parameters and methods:
'''
    1. feature detection method:
        - SIFT
        - SURF
        - ORB
    2. feature matching method:
        - Brute Force
        - FLANN
    3. feature matching ratio:
        - 0.7
        - 0.8
        - 0.9
    
'''
f_detector = {
    'SIFT': cv2.SIFT_create(),
    #'SURF': cv2.xfeatures2d.SURF_create(),  # this one is patented
    'ORB': cv2.ORB_create()
}

f_matcher = {
    'BF': cv2.BFMatcher(),
    'FLANN': cv2.FlannBasedMatcher()
}

f_ratio = [0.7, 0.8, 0.9]

In [4]:
# grid search in parallel
import concurrent.futures
import itertools

# create a list of all the combinations
all_combinations = list(itertools.product(f_detector.keys(), f_matcher.keys(), f_ratio))

# perform the grid search in parallel
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(cal_psnr_ssim, itertools.repeat(files), 
                           [f_detector[c[0]] for c in all_combinations], 
                           [f_matcher[c[1]] for c in all_combinations], 
                           [c[2] for c in all_combinations], 
                           itertools.repeat(num))
    
    # print the results
    for comb, res in zip(all_combinations, results):
        print(comb, res)

TypeError: cannot pickle 'cv2.SIFT' object

In [5]:
print(f_detector.keys())

dict_keys(['SIFT', 'ORB'])
